# Basic membrane representation

In [2]:
import numpy as np
from numba import cuda
from collections import defaultdict

In [3]:
EVOLVE = 0
SENDIN = 1
SENDOUT = 2
DISSOL = 3
DIVIDE = 4

In [98]:
class PMembrane(object):
    def __init__(self, O=[], multiset=''):
        self.O = O                                    # Alphabet
        self.H = [1]                                  # Label of membranes (first is the skin) (level in this case)
        self.mu = [-1]                                # Tree with indices (membranes will be 0-indexed)
                                                      # mu: membrane -> parent
        self.omega = [multiset]                       # Multiset of objects placed in the regions of mu

#         self.rules_evolution = defaultdict(list)      # (level, object) -> object : [o]l => [o]l
#         self.rules_sendin = defaultdict(list)         # (level, object) -> object : o []l => [o]l
#         self.rules_sendout = defaultdict(list)        # (level, object) -> object : [o]l => []l o
#         self.rules_dissolution = defaultdict(list)    # (level, object) -> object : [o]l => o
#         self.rules_division = defaultdict(list)       # (level, object) -> object : [o]l => [o]l [o]l
        
        self.rules = [defaultdict(list) for i in range(5)]
        # [evolution, sendin, sendout, dissolution, division]
        # Structure is shown in commented code above

    def add_membrane(self, parent, label, multiset=''):
        '''Adds a membrane of label / layer and with multiset inside a parent membrane'''
        self.H.append(label)
        self.mu.append(parent)
        self.omega.append(multiset)
        
    def add_rule(self, label, obj, ruletype, new_obj):
        '''Adds a rule for a membrane label and an object'''
        self.rules[ruletype][(label, obj)] = new_obj
        
    def level2_membranes(self):
        '''Returns a list of level 2 membranes'''
        return [ndx for ndx, parent in enumerate(self.mu) if parent == 0]
    
    def level3_membranes(self):
        '''Returns a list of level 3 membranes'''
        return [ndx+1 for ndx, parent in enumerate(self.mu[1:]) if self.mu[parent] == 0]
    
    def level2_submembranes_multisets(self, membrane):
        '''Returns a count matrix of submembranes x objects'''
        '''The matrix will serve as input to the kernel'''
        if self.mu[membrane] == 0:
            
            # Get list of submembranes
            submembranes = [ndx for ndx, parent in enumerate(self.mu) if parent == membrane]
            matrix = np.zeros((len(submembranes), len(self.O)),dtype=np.int16)
            
            for ndx, sm in enumerate(submembranes):
                for obj in self.omega[sm]:
                    matrix[ndx, self.O.index(obj)] += 1

            return matrix

    def level2_submembranes(self, membrane):
        '''Returns the specific membranes in the count matrix'''
        if self.mu[membrane] == 0:

            return [ndx for ndx, parent in enumerate(self.mu) if parent == membrane]
        
    def level2_multiset(self, membrane):
        '''Returns an array of object counts in the multiset of the level 2 membrane'''
        if self.mu[membrane] == 0:
            matrix = np.zeros(len(self.O), dtype=np.int16)
            for obj in self.omega[membrane]:
                matrix[self.O.index(obj)] += 1
            
            return matrix
        
    def ruleset(self):
        '''
        Returns an array of tuples, each one representing a rule:
            (object, membrane, ruletype, output_object)
        The array is sorted, and an array of indices will also be returned
            indices[obj] = ndx where the thread will start looking for rules
        For simplicity, division rules will only output 2 membranes with the same evolved object'''
        ruleset = np.zeros((0,4), dtype=np.int16)
        for i in range(5):
            for k, v in self.rules[i].items():
                ruleset = np.vstack((ruleset, [
                    self.O.index(k[1]),
                    k[0],
                    i,
                    self.O.index(v)
                ]))
        ruleset = ruleset[np.lexsort(np.rot90(ruleset))]
        objects = [r[0] for r in ruleset]
        indices = np.array([objects.index(i) if i in objects else -1 for i in range(len(self.O))], dtype=np.int16)
        return ruleset, indices

In [99]:
# Test membrane

test = PMembrane(O=['a', 'b', 'c'], multiset="a")
test.add_membrane(0, 2, multiset="abc")
test.add_membrane(0, 2, multiset="aab")
test.add_membrane(0, 2, multiset="abcc")
test.add_membrane(1, 3, multiset="a")
test.add_membrane(1, 3, multiset="bb")
test.add_membrane(1, 3, multiset="c")
test.add_membrane(2, 3, multiset="ac")
test.add_membrane(2, 3, multiset="bbc")
test.add_membrane(2, 3, multiset="ccc")
test.add_membrane(3, 3, multiset="aa")
test.add_membrane(3, 3, multiset="bc")
test.add_membrane(3, 3, multiset="b")

test.add_rule(3, 'a', EVOLVE, 'b')
test.add_rule(3, 'b', EVOLVE, 'c')
test.add_rule(2, 'a', EVOLVE, 'b')
test.add_rule(2, 'b', EVOLVE, 'c')

In [100]:
test.level2_membranes()

[1, 2, 3]

In [101]:
test.level2_submembranes_multisets(1)

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 1]], dtype=int16)

In [102]:
test.level3_membranes()

[4, 5, 6, 7, 8, 9, 10, 11, 12]

In [103]:
test.level2_multiset(2)

array([2, 1, 0], dtype=int16)

In [104]:
# (object, membrane, ruletype, output object)
test.ruleset()

(array([[0, 2, 0, 1],
        [0, 3, 0, 1],
        [1, 2, 0, 2],
        [1, 3, 0, 2]]), array([ 0,  2, -1], dtype=int16))

# Selection / Evolution Kernel

In [105]:
@cuda.jit
def selection_kernel(M_3, M_2, R, R_ndx, out, rulecount):
    obj = cuda.threadIdx.x
    mem_3 = cuda.threadIdx.y
    mem_2 = cuda.blockIdx.x
    
    # TODO: flag to see if any rules were selected

    # If rules for this object exist and if this object is in the multiset of this membrane:
    if ((R_ndx[obj] >= 0) and (M_3[mem_2][mem_3][obj] > 0)):
        # Iterate through all the relevant rules
        for i in range(R_ndx[obj], rulecount):
            # Once you encounter a rule for a new object, you're done
            if (R[i][0] != obj):
                break
            # Select Level 2 rules as well, if the object is in its multiset
            if ((R[i][1] == 2) and (M_2[mem_2][obj] > 0)):
                out[i] = 1
            # Only select 1 rule, then leave
            if (R[i][1] == 3):
                out[i] = 1
                break

In [106]:
R, R_ndx = test.ruleset()
out = np.zeros(len(R), dtype=np.int16)

mems_2 = test.level2_membranes()
M_3 = np.dstack(
    (test.level2_submembranes_multisets(i) for i in mems_2)
)
M_2 = np.vstack(
    (test.level2_multiset(i) for i in mems_2)
)

In [107]:
blocks_per_grid = 3
threads_per_block = (3,3)
selection_kernel[blocks_per_grid, threads_per_block](M_3, M_2, R, R_ndx, out, len(R))

In [108]:
out

array([1, 1, 1, 1], dtype=int16)